In [1]:
#DiamondCalc.py created 2/27/2023 by BWM and EGM
#program to calculate heat load and current in diamond devices

import numpy as np
import os
import matplotlib.pyplot as plt
import math
cwd = os.getcwd()

##############################################
#Get elemental data in and properly formed
##############################################
carbon = np.loadtxt((cwd + "/sourceData/carbon.txt"), dtype = float,comments ="#",)
pt = np.loadtxt((cwd + "/sourceData/pt.txt"), dtype = float,comments ="#",)
al = np.loadtxt((cwd + "/sourceData/al.txt"), dtype = float,comments ="#",)
###############################################
# Constants
###############################################
densC = 3.15 #g/cm^3
densPt = 21.4 #g/cm^3
densAl = 2.7 #g/cm^3
enConv = 1.60218e-19 #eV per Joule
coulPerElec =  6.28e18  #elec per s for amp
micro_cm = 1e-4 # microns to cm conversion
nano_cm = 1e-7 # nm to cm conversion
ToMev= 1e-6
freeCarrier = 13.2 #energy to make electorn whole pairs in diamond
################################################


def MAdevice_perf_al (diamon_thick, metal_thick,energy, flux):
    bias = 10 #bias on dimaond
    coefAl = np.interp((energy*ToMev), al[:,0], al[:,2])
    coefC = np.interp((energy*ToMev), carbon[:,0], carbon[:,2])
    transAl = (math.exp(-coefAl*metal_thick*nano_cm*densPt))
    ptLayerAbs = (flux * (1-transAl))
    flux_to_diamond =flux-ptLayerAbs
    transC = (math.exp(-coefC*diamon_thick*micro_cm*densC))
    cLayerAbs = flux_to_diamond * (1-transC)
    current = (cLayerAbs * energy * enConv/ (freeCarrier))
    absHeat = (ptLayerAbs * 2 + cLayerAbs) * energy * enConv
    jouleHeat = current * bias
    heatLoad = absHeat + jouleHeat 
    absAlall = (1-transAl) * 2
    absC = 1-transC
    return current , heatLoad, absAlall, absC

def MAdevice_perf_pt (diamon_thick, metal_thick,energy, flux):
    bias = 10 #bias on dimaond
    coefC = np.interp((energy*ToMev), carbon[:,0], carbon[:,2])
    coefPt = np.interp((energy*ToMev), pt[:,0], pt[:,2])
    transPt = (math.exp(-coefPt*metal_thick*nano_cm*densPt))
    ptLayerAbs = (flux * (1-transPt))
    flux_to_diamond =flux-ptLayerAbs
    transC = (math.exp(-coefC*diamon_thick*micro_cm*densC))
    cLayerAbs = flux_to_diamond * (1-transC)
    current = (cLayerAbs * energy * enConv/ (freeCarrier))
    absHeat = (ptLayerAbs * 2 + cLayerAbs) * energy * enConv
    jouleHeat = current * bias
    totalHeat = absHeat + jouleHeat 
    absPtall = (1-transPt) * 2
    absC = 1-transC
    return current , totalHeat, absPtall, absC, absHeat




   



## How to use
MAdevice_perf_pt returns a tuple. 
- 0 = current
- 1 = heatload 
- 2 = absorbtion in both metal layers
- 3 = absorbtion in diamond

In [8]:
totFlux = 1.6e16
steps = 11
totHeat = 0
for eneg in range(steps):
    eneV = (eneg + 5)*1000
    flux = totFlux/steps
    MADevice = MAdevice_perf_pt(45,30,eneV,flux)
    totHeat = totHeat + (MADevice[1])

print( "total load on diamond us {}".format (totHeat) ) 
    
        


total load on diamond us 2.539498905748913


In [ ]:
list2 = {4.8:1e25,7:1e15,30:1e13, 9:1e15}
for eneg in list2:
        eneV = eneg*1000
        flux = list2[eneg]
        MADevice = MAdevice_perf_pt(45,30,eneV,flux)
        totPow = (MADevice[2]+MADevice[3]) * eneV * enConv
        
        print ("for Energy {} keV with {:1.2e} ph/s \nExpected Currnet {:1.2e} A, Expected Heat {:1.2e} W, Absorbed energy in Device {:1.2e} W\n".format(eneg, flux,MADevice[0], MADevice[1], MADevice[4]))

In [ ]:

eneV = 3000
flux = 2e15
MADevice = MAdevice_perf_pt(45,30,eneV,flux)
totPow = (MADevice[2]+MADevice[3]) * eneV * enConv

print ("for Energy {} keV with {:1.2e} ph/s \nExpected Currnet {:1.2e} A, Expected Heat {:1.2e} W, Absorbed energy in Device {:1.2e} W\n".format(eneV, flux,MADevice[0], MADevice[1], MADevice[4]))